# <a id='toc1_'></a>[create dataset](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [create dataset](#toc1_)    
  - [init code](#toc1_1_)    
  - [load data](#toc1_2_)    
    - [option clean csv](#toc1_2_1_)    
    - [option dirty csv](#toc1_2_2_)    
  - [arrange data](#toc1_3_)    
    - [trim column names](#toc1_3_1_)    
    - [set date column](#toc1_3_2_)    
    - [remove duplicates or invalids](#toc1_3_3_)    
    - [parse dates to datetime 🕥](#toc1_3_4_)    
  - [reduce data](#toc1_4_)    
    - [(optional) remove columns w/ no or few values](#toc1_4_1_)    
  - [condensed dataset](#toc1_5_)    
    - [define single, multi and misc columns](#toc1_5_1_)    
    - [create condensed dataset](#toc1_5_2_)    
    - [create manual table for mapping](#toc1_5_3_)    
    - [apply new column names from mapping table](#toc1_5_4_)    
  - [💾 data for pivot use](#toc1_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[init code](#toc0_)

In [30]:

import io
import os
from glob import glob
import pandas as pd
from pathlib import Path
from pandas_plots import tbl, pls, hlp
from pandas_plots.hlp import add_bitmask_label
import re
hlp.show_package_version(["pygwalker"])

os.environ["THEME"]="dark"


dir_data_in=Path("../data/in")
dir_data_out=Path("../data/out")

# ! only 1 csv file in in/ is allowed
file_csv_list=glob(f"{dir_data_in}/*.csv")
assert len(file_csv_list) == 1, f"❌ Expected 1 csv file, found {len(file_csv_list)}"

file_csv = file_csv_list[0]

ENABLE_UPDATE=True

REGEX_CHOICE = r"( \(choice=.*?\)+|\.\d$)"
REGEX_CODE = r"(^\[.*?\])"
REGEX_NONCODE = r"\].*"

🐍 3.12.2 | 📦 pygwalker: 0.4.9.13 | 📦 pandas: 2.2.3 | 📦 numpy: 1.26.4 | 📦 duckdb: 1.2.0 | 📦 pandas-plots: 0.12.14 | 📦 connection_helper: 0.8.13


## <a id='toc1_2_'></a>[load data](#toc0_)


### <a id='toc1_2_1_'></a>[option clean csv](#toc0_)

In [31]:
df_csv = pd.read_csv(file_csv, sep=",", encoding="utf-8-sig", dtype={"[01.01] Bewertung - CTCAE-Bewertung":str})

### <a id='toc1_2_2_'></a>[option dirty csv](#toc0_)
- ⚠️ strategy to import the csv raw data:
  - ❌ do not use default `pd.read_csv()`, its not as error tolerant as excel
  - ✅ read csv as `windows-1252` string and cleanse it
    - most (not all) lines have heading + trailing quotes, remove
    - data have a mixture of "" and ", downgrade by 1
    - delimiter "," also occurs in quotes, change to ";" only outside of quotes
    - now byte stream can be imported

In [32]:
%%script echo skipping

# * replace delimiter *only* outside of quotes
def replace_delimiter_outside_quotes(
    input: str, delimiter_old: str = ",", delimiter_new: str = ";", quotechar: str = '"'
):
    outside_quotes = True
    output = ""
    # * loop through input and toggle inside/outside status
    for char in input:
        if char == quotechar:
            outside_quotes = not outside_quotes
        elif outside_quotes and char == delimiter_old:
            char = delimiter_new
        output += char
    return output

# * downgrades quotes in string to normalize quots: " -> NA, "" -> "
def reduce_quotes_in_string(input: str):
    output = ""
    len_=len(input)
    i = 0
    while i < len_:
        # * check current + next char if within range
        if input[i] == '"' and i+1 < len_ and input[i+1] != '"':
            output += ''
            i = i+1
        elif input[i] == '"'and i+1 < len_ and input[i+1] == '"':
            output+= '"'
            i = i+2
        else:
            output += input[i]
            i=i+1
    return output

# * read csv -> variable
with open(file_csv, encoding="windows-1252") as f:
    csv=f.read()

# * selectively change , into ;
csv_lines = csv.splitlines()
csv_lines_new = []
for line in csv_lines[:]:
    # * remove first and last quotes if present
    if line[:1] == '"':
        line = line[1:-1]
    # * tailor line by line
    line=reduce_quotes_in_string(line)
    line=replace_delimiter_outside_quotes(line)
    # * just now quotes can be removed, not before
    line=line.replace('"', '')
    csv_lines_new.append(line)
    # print(f'{line.count(";")} {line}')

# * list -> string fo feed into df constructor
csv_string = "\n".join(csv_lines_new).replace('\t', '')

df_csv = pd.read_csv(
    io.StringIO(csv_string),
    sep=";",
    encoding="windows-1252",
)

skipping


## <a id='toc1_3_'></a>[arrange data](#toc0_)

### <a id='toc1_3_1_'></a>[trim column names](#toc0_)

In [33]:
cols_trimmed = [col.strip() for col in df_csv.columns]
df_csv.columns = cols_trimmed

### <a id='toc1_3_2_'></a>[set date column](#toc0_)

In [34]:
[print(col) for col in df_csv.columns if "Datum" in col]
col_date = "[02.01] Basics - Datum"

[02.01] Basics - Datum


### <a id='toc1_3_3_'></a>[remove duplicates or invalids](#toc0_)

In [35]:
# # * remove known erroruos lines
df_csv.drop([0,34,33], inplace=True, axis=0, errors='raise')
# * remove "löschen" lines
df_csv = df_csv[~(df_csv[col_date] == "löschen")]
df_csv

,Participant ID,Survey Identifier,Survey Timestamp,[02.01] Basics - Datum,[01.01] Bewertung - CTCAE-Bewertung,[01.02] Wann ist das AE erstmals aufgetreten?,"[01.03] Bewertung - Glauben Sie, dass das AE mit dem Sport assoziiert ist?",[02.02] Basics - Was war das AE? (choice=Schmerzen),[02.02] Basics - Was war das AE? (choice=Übelkeit/ Erbrechen),[02.02] Basics - Was war das AE? (choice=Oberflächliche Verletzungen),...,Welche motorische Hauptform dominierte die Bewegungseinheit ? (choice=Schnelligkeit).1,In welcher Phase der geplanten Sporteinheit trat das AE auf?.1,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Gut).1,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Durchschnittlich).1,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Mäßig).1,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Weiß nicht).1,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Anmerkung).1,"Glaubst Du, dass das AE mit dem Sport assoziiert ist?",Anmerkung.1,Complete?.2
1,3,NaN,2022-10-13 13:41:16,2022_10,2,Das AE war vor dem Sport bereits vorhanden und...,No,Unchecked,Checked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
2,4,NaN,2022-11-08 09:19:23,2022_10,1,Das AE ist neu aufgetreten.,Yes,Checked,Unchecked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
3,5,NaN,2022-11-08 09:31:05,2022_11,2,Das AE war vor dem Sport bereits vorhanden und...,No,Unchecked,Unchecked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
4,6,NaN,2022-11-18 10:10:55,2022_11,1,Das AE ist neu aufgetreten.,No,Unchecked,Checked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
5,7,NaN,2022-11-21 14:02:25,2022_11,3,Das AE ist neu aufgetreten.,Yes,Unchecked,Unchecked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,204,NaN,NaN,2024_11_27,1,Das AE ist neu aufgetreten.,Yes,Unchecked,Unchecked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
194,205,NaN,NaN,2024_12_03,1,Das AE war vor dem Sport bereits vorhanden und...,Yes,Unchecked,Unchecked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
195,206,NaN,NaN,2024_12_08,2,Das AE ist neu aufgetreten.,Yes,Unchecked,Checked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
196,207,NaN,NaN,2024_12_12,1,Das AE ist neu aufgetreten.,Yes,Checked,Unchecked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN


### <a id='toc1_3_4_'></a>[parse dates to datetime 🕥](#toc0_)

In [36]:
# %%script echo skipping

# * manual override of invalid dates
df_csv.loc[df_csv[col_date]=="2023_08_2028", col_date]="2023_08_28"
df_csv.loc[df_csv[col_date]=="2024_16", col_date]="2024_06"

# * split column into  all possible date patterns, these may not interfere!
# * parse date col
REG_DATE_YYYY_MM=r"\d{4}_\d{2}$"
REG_DATE_YYYY_MM_DD=r"\d{4}_\d{2}_\d{2}$"
REG_DATE_DD_MM_YYYY=r"\d{2}.\d{2}.\d{4}$"

# * get subsets according to regex
_df1=df_csv[df_csv[col_date].str.contains(REG_DATE_YYYY_MM, na=False)][col_date]
_df2=df_csv[df_csv[col_date].str.contains(REG_DATE_YYYY_MM_DD, na=False)][col_date]
_df3=df_csv[df_csv[col_date].str.contains(REG_DATE_DD_MM_YYYY, na=False)][col_date]

# * convert each to corresponding datetime
_df1=pd.to_datetime(_df1, format="%Y_%m").astype(str)
_df2=pd.to_datetime(_df2, format="%Y_%m_%d").astype(str)
_df3=pd.to_datetime(_df3, format="%d.%m.%Y").astype(str)

# * update in place
df_csv.update(_df1)
df_csv.update(_df2)
df_csv.update(_df3)

# * now convert the whole col
df_csv[col_date]=pd.to_datetime(df_csv[col_date], format='ISO8601')

# * blend timestamp into date column
df_csv[col_date].fillna(df_csv["Survey Timestamp"], inplace=True)

## <a id='toc1_4_'></a>[reduce data](#toc0_)

### <a id='toc1_4_1_'></a>[(optional) remove columns w/ no or few values](#toc0_)

In [37]:
# import numpy as np
# # * remove columns with no value
# cols_no_value = [col for col in df_csv.columns if df_csv[col].isna().all()]
# print("these columns have no values at all:")

# print(np.array(cols_no_value))

# # ! remove columns
# cols_after_removal = list(set(df_csv.columns) - set(cols_no_value))
# df_csv_short = df_csv[list(set(df_csv.columns) - set(cols_no_value))]

In [38]:

def condense_columns_single_choice(df, strict=False):
    """
    Condenses multiple columns into a single target column, extracting specific values based on
    the first occurrence of 'Checked' in each row. 

    The function processes columns named according to a pattern (e.g., 'choice=<value>') and condenses
    them into a single column. If a column contains the value 'Checked', the corresponding value (e.g., 
    'Schmerzen', 'Muskelkater') is assigned to the new target column. If multiple 'Checked' values are 
    present, only the leftmost occurrence is used.

    Parameters:
        df (pandas.DataFrame): The input DataFrame with columns to be processed. The columns should contain
                                values such as 'Checked' or 'Unchecked', and be named according to a 
                                pattern like '[02.02] Basics - Was war das AE?  (choice=<value>)'.
        strict: if an error is raised if more than 1 cell is "Checked"

    Returns:
        pandas.DataFrame: A DataFrame with the same columns as input, but with a new target column containing 
                        the extracted values based on the first occurrence of 'Checked' in each row.
                        The new column is named using the pattern found in the column names (e.g., '[02.02]').

    Notes:
        - If no 'Checked' value is found in a row, the target column is set to pd.NA for that row.
        - The new column name is derived from the first part of the first input column (i.e., the substring
        within the square brackets).
        - Only the first 'Checked' occurrence in each row is considered; any subsequent 'Checked' cells
        will be ignored.
    """

    # * Extract target column name using the updated regex pattern (substring in square brackets)
    match = re.search(r"\[\d{2}\.\d{2}(\.\d{2})?\]", df.columns[0])
    if not match:
        raise ValueError("No substring in square brackets matching the pattern found in the first column name.")
    target_col_name = match.group(0)

    # * Check for multiple "checked" cells in one row
    columns_to_check = [col for col in df.columns if re.search(r"\(choice=.*\)", col)]
    checked_counts = df[columns_to_check].apply(lambda row: (row.str.lower() == "checked").sum(), axis=1)

    if strict and (checked_counts > 1).any():
        problematic_rows = df.index[checked_counts > 1].tolist()
        raise ValueError(f"Multiple 'checked' values found in rows: {problematic_rows}")

    # * Create the target column with default pd.NA values
    df[target_col_name] = pd.NA

    # * Process each row and select the first "checked" value from left to right
    for index, row in df.iterrows():
        # * Loop over columns left to right to find the first "checked" value
        for col in columns_to_check:
            if row[col].lower() == "checked":
                # * Extract the choice label using regex. capture group is 
                match = re.search(r"\(choice=(.*?)\)", col)
                # match = re.search(r"\(choice=\d+ (.*?)\)", col)
                if match:
                    choice_label = match.group(1)
                    # Set the target column with the choice label for this row
                    df.at[index, target_col_name] = choice_label
                break  # Stop after the first "checked" value is found

    return df.iloc[:,-1]

def condense_columns_multi_choice(
    df, 
    new_col_pattern=r"^\[.*?\]", 
    choice_pattern=r"\(choice=(.*?)\)"
):
    # Extract the new column name from the first column using the provided regex pattern
    first_col = df.columns[0]
    new_col_name = re.search(new_col_pattern, first_col).group(0) if re.search(new_col_pattern, first_col) else "Unnamed"
    
    # Extract parts from original column names using the provided choice pattern
    extracted_parts = [
        re.search(choice_pattern, col).group(1) if re.search(choice_pattern, col) else None
        for col in df.columns
    ]

    # Calculate the new column using a bitmask
    def calculate_bitmask(row):
        bitmask = 0
        for i, col in enumerate(df.columns):
            if row[col] == "Checked":
                bitmask += 2**i
        return bitmask

    df[new_col_name] = df.apply(calculate_bitmask, axis=1)
    
    # Example placeholder for a function to add additional labels or transformations
    # Replace `add_bitmask_label` with your actual logic if needed
    # add_bitmask_label(df, new_col_name, extracted_parts)

    add_bitmask_label(df, new_col_name, extracted_parts)
    return df.iloc[:,-1]


## <a id='toc1_5_'></a>[condensed dataset](#toc0_)

### <a id='toc1_5_1_'></a>[define single, multi and misc columns](#toc0_)

In [63]:

# todo: duplicate: '[03.05.01]'

stubs_single = set((
'[03.01.02]',
'[03.14.02]', '[03.15.01]',
))


stubs_multi = set((
'[02.02]', '[02.03]', '[02.04]', '[05.07]', '[05.09]',
'[03.03.02]', '[03.03.01]', '[03.04.01]', '[03.06.01]',
'[03.07.01]', '[03.08.01]', '[03.11.01]', '[03.12.01]', '[03.14.01]',
'[03.17.01]',
))

stubs_misc= set((
'Participant ID',
'[01.02]','[01.03]', '[03.01]', '[03.02]', '[03.03]',
'[03.04]', '[03.05]', '[03.06]', '[03.07]', '[03.08]', '[03.09]', '[03.10]',
'[03.11]', '[03.12]', '[03.13]', '[03.15]', '[03.16]', '[03.17]', '[05.01]',
'[05.02]', '[05.03]', '[05.04]', '[05.05]', '[05.06]', '[05.08]',
'[03.01.01]', '[03.02.01]',
'[03.02.02]',
'[03.04.02]', '[03.05.02]', '[03.06.02]', '[03.07.02]', '[03.08.02]',
'[03.09.01]', '[03.09.02]', '[03.10.01]', '[03.10.02]', '[03.11.02]',
'[03.12.02]', '[03.13.01]', '[03.13.02]', 
'[03.15.02]', '[03.17.02]',

'[02.01]', '[01.01]', '[01.02]', '[03.01]',
'[03.02]', '[03.03]', '[03.04]', '[03.07]',
'[03.07.02] ', '[03.08]', '[03.08.02]',
'[03.09]', '[03.09.01]', '[03.09.02]', '[03.10]', '[03.11]', '[03.12]',
'[03.13]', '[03.13.01]', '[03.13.02]', '[03.17]', '[05.01]', '[05.02]',
'[05.03]', '[05.04]', '[05.05]', '[05.06]', '[05.08]',
'[06.01]', '[07.07]', '[03.18]', '[07.01]', '[07.04]',
))

if stubs_misc & stubs_multi or stubs_misc & stubs_single or stubs_single & stubs_multi:
    raise ValueError("stubs must be disjoint")

cols_single = hlp.find_cols(df_csv.columns, stubs_single)
cols_multi = hlp.find_cols(df_csv.columns, stubs_multi)
cols_misc = hlp.find_cols(df_csv.columns, stubs_misc)


In [64]:
df_csv.find_cols(stubs=[" alt"])

['[05.03] Rahmenbedingungen - Wie alt war der/die Betroffene zum Zeitpunkt des AE?',
 'Wie alt war der/die Betroffene zum Zeitpunkt des AE?',
 'Wie alt war der/die Betroffene zum Zeitpunkt des AE?.1']

In [65]:
df_csv_single = pd.DataFrame()
for stub in stubs_single:
    cols = [col for col in df_csv.columns if stub in col]
    print(cols)

    df = pd.DataFrame(df_csv[cols])
    ser = condense_columns_single_choice(df)
    df_csv_single[stub] = ser

df_csv_single

['[03.15.01] Folgen - War die gesamte Handlungsweise bei diesem AE rein pädagogischer Natur? (choice=ja)', '[03.15.01] Folgen - War die gesamte Handlungsweise bei diesem AE rein pädagogischer Natur? (choice=nein)']
['[03.14.02] Folgen - Wie lange wurde die Anpassung vorgenommen? (choice=nur für diese Einheit)', '[03.14.02] Folgen - Wie lange wurde die Anpassung vorgenommen? (choice=für die gesamte Therapiephase)', '[03.14.02] Folgen - Wie lange wurde die Anpassung vorgenommen? (choice=ab jetzt für alle bewegungseinheiten mit allen Patient*innen)']
['[03.01.02] Folgen - Wie lange dauerten die Schmerzen an? (choice=Schmerzen am gleichen Tag)', '[03.01.02] Folgen - Wie lange dauerten die Schmerzen an? (choice=Schmerzen bis zum Folgetag)', '[03.01.02] Folgen - Wie lange dauerten die Schmerzen an? (choice=Schmerzen, die mindestens 3 Tage anhielten)', '[03.01.02] Folgen - Wie lange dauerten die Schmerzen an? (choice=weiß nicht)']


,[03.15.01],[03.14.02],[03.01.02]
1,<NA>,<NA>,<NA>
2,<NA>,nur für diese Einheit,Schmerzen am gleichen Tag
3,<NA>,<NA>,<NA>
4,<NA>,<NA>,<NA>
5,ja,<NA>,"Schmerzen, die mindestens 3 Tage anhielten"
...,...,...,...
193,<NA>,<NA>,<NA>
194,<NA>,<NA>,<NA>
195,<NA>,<NA>,<NA>
196,<NA>,<NA>,<NA>


In [66]:
df_csv_multi = pd.DataFrame()
for stub in stubs_multi:
    cols = [col for col in df_csv.columns if stub in col]
    # print(cols)

    df = pd.DataFrame(df_csv[cols])
    ser = condense_columns_multi_choice(df)
    df_csv_multi[stub] = ser

df_csv_multi[:3]

,[05.07],[05.09],[03.03.02],[03.06.01],[03.04.01],[02.03],[03.08.01],[03.14.01],[03.17.01],[03.03.01],[03.11.01],[02.04],[03.07.01],[02.02],[03.12.01]
1,Koordination,2 Durchschnittlich,-,-,-,Physische Belastung|Med. Therapie,bei dem Betroffenen selbst,-,-,-,-,Internistisch,-,Übelkeit/ Erbrechen,-
2,Koordination,1 Gut,-,-,-,Koordinationsprobleme,bei dem Betroffenen selbst|im Behandlungsteam,Übungsauswahl|Intensität,-,-,-,Rücken|Gesäß,-,Schmerzen,-
3,Koordination,3 Mäßig|5 Anmerkung,Innerhalb der ersten 24 Stunden,-,-,Med. Therapie,bei dem Betroffenen selbst,-,-,Medikamentengabe oral,-,Internistisch,-,Kreislaufprobleme,-


### <a id='toc1_5_2_'></a>[create condensed dataset](#toc0_)

In [67]:
import re

# * save original columns as txt
cols_all = sorted(cols_single + cols_multi + cols_misc)
pd.DataFrame(cols_all).to_csv(dir_data_out/"df_csv_condensed_columns.txt", index=False, sep=";", header=False)

# * all columns
df_csv_condensed = pd.concat(
    [
        df_csv[cols_misc],
        df_csv_single,
        df_csv_multi,
    ], 
    axis=1,
)
df_csv_condensed[:3]

,Participant ID,[02.01] Basics - Datum,[01.01] Bewertung - CTCAE-Bewertung,[01.02] Wann ist das AE erstmals aufgetreten?,"[01.03] Bewertung - Glauben Sie, dass das AE mit dem Sport assoziiert ist?","[06.01] Freitext - Basics - Genaue Beschreibung zur Entstehung, des Auslösers und zur Art des AE.",[03.01] Folgen - Sind Schmerzen aufgetreten?,[03.01.01] Folgen - Schmerzskala,[03.02] Folgen - Kam es zu einem Krankenhausaufenthalt oder einer Verlängerung eines solchen?,[03.02.01] Folgen - Wie zeitnah anch dem AE wurde entschieden einen Krankehausaufenthalt anzustreben oder zu verlängern?,...,[02.03],[03.08.01],[03.14.01],[03.17.01],[03.03.01],[03.11.01],[02.04],[03.07.01],[02.02],[03.12.01]
1,3,2022-10-01,2,Das AE war vor dem Sport bereits vorhanden und...,No,Beim moderaten Tischtennisspielen wurde der Pa...,Nein,NaN,No,NaN,...,Physische Belastung|Med. Therapie,bei dem Betroffenen selbst,-,-,-,-,Internistisch,-,Übelkeit/ Erbrechen,-
2,4,2022-10-01,1,Das AE ist neu aufgetreten.,Yes,Luftballonspiel im Stehen; Kind hat sich gestr...,Ja,2.0,No,NaN,...,Koordinationsprobleme,bei dem Betroffenen selbst|im Behandlungsteam,Übungsauswahl|Intensität,-,-,-,Rücken|Gesäß,-,Schmerzen,-
3,5,2022-11-01,2,Das AE war vor dem Sport bereits vorhanden und...,No,Beim leichter Mobilisation und Aktivierung hat...,Nein,NaN,No,NaN,...,Med. Therapie,bei dem Betroffenen selbst,-,-,Medikamentengabe oral,-,Internistisch,-,Kreislaufprobleme,-


### <a id='toc1_5_3_'></a>[create manual table for mapping](#toc0_)

In [68]:
import re

# * have all columns but remove choices from column names, isolate code
df_csv_cols = (
    pd.DataFrame(df_csv.columns, columns=["col"])
    .assign(
        col_code=lambda x: x["col"].str.extract(REGEX_CODE),
        col_clean=lambda x: x["col"].str.replace(REGEX_CHOICE, "", regex=True),
    )
)
display(df_csv_cols[:3])

# * group and set up foe manual renaming
df_csv_cols_condensed = (df_csv_cols
    .assign(cnt=1)
    .groupby(["col_clean","col_code"], as_index=False, dropna=False)["cnt"].sum()
)
display(df_csv_cols_condensed[:3])

if ENABLE_UPDATE and False:
    df_csv_cols_condensed.assign(col_new=lambda x: x["col_code"]).to_excel(dir_data_out/"df_csv_cols.xlsx", index=False)
    # df_csv_cols_condensed.assign(col_new=lambda x: x["col_code"]).to_excel(dir_data_out/"df_csv_cols.ods", index=False, engine="odf")


,col,col_code,col_clean
0,Participant ID,NaN,Participant ID
1,Survey Identifier,NaN,Survey Identifier
2,Survey Timestamp,NaN,Survey Timestamp


,col_clean,col_code,cnt
0,An welchem NAOK-Standort ist das AE aufgetreten?,NaN,2
1,Anmerkung,NaN,2
2,Bei welchem Anlass trat das AE auf?,NaN,1


### <a id='toc1_5_4_'></a>[apply new column names from mapping table](#toc0_)

In [69]:
# # * import column mapping
df_csv_cols_import = pd.read_excel(dir_data_in/"df_csv_cols.xlsx")#.query("col_new.notna()")
# * coalesce columns - most right column has priority
df_csv_cols_import["col_final"] = df_csv_cols_import[["col_new", "col_code", "col_clean"]].backfill(axis=1).iloc[:,0]

# * create dict for mapping
# * column names in target df can be full label or code only, so 2 rounds are needed
def create_col_map(df, join_col: str):
    # * use concat to join two series
    return (pd.concat(
        [
            df_csv_condensed.columns.to_series(),
            # * only use rows with not NA values
            df_csv_cols_import[df_csv_cols_import[join_col].notna()].set_index(join_col)["col_final"],
        ],
        axis=1, join="inner"
        )
        ["col_final"]
        .to_dict()
    )

col_map1 = create_col_map(df_csv_cols_import, "col_clean")
col_map2 = create_col_map(df_csv_cols_import, "col_code")

# ! relabel column names in 2 rounds
df_csv_condensed.rename(columns=col_map1, inplace=True)
df_csv_condensed.rename(columns=col_map2, inplace=True)

# ! or strip column names to code only
# cols = df_csv_condensed.columns
# df_csv_condensed.columns = [re.sub(REGEX_NONCODE, "]", col) for col in cols]

df_csv_condensed[:3]

,Participant ID,[02.01] Datum,[01.01] CTCAE,[01.02] Ab wann,[01.03] Sportassoziiert?,[06.01],[03.01] Schmerzen?,[03.01.01] Schmerzskala,[03.02] KH?,[03.02.01] Wann KH entschieden,...,[02.03] Auslöser,[03.08.01],[03.14.01],[03.17.01],[03.03.01],[03.11.01],[02.04] Körperteil,[03.07.01],[02.02] AE,[03.12.01]
1,3,2022-10-01,2,Das AE war vor dem Sport bereits vorhanden und...,No,Beim moderaten Tischtennisspielen wurde der Pa...,Nein,NaN,No,NaN,...,Physische Belastung|Med. Therapie,bei dem Betroffenen selbst,-,-,-,-,Internistisch,-,Übelkeit/ Erbrechen,-
2,4,2022-10-01,1,Das AE ist neu aufgetreten.,Yes,Luftballonspiel im Stehen; Kind hat sich gestr...,Ja,2.0,No,NaN,...,Koordinationsprobleme,bei dem Betroffenen selbst|im Behandlungsteam,Übungsauswahl|Intensität,-,-,-,Rücken|Gesäß,-,Schmerzen,-
3,5,2022-11-01,2,Das AE war vor dem Sport bereits vorhanden und...,No,Beim leichter Mobilisation und Aktivierung hat...,Nein,NaN,No,NaN,...,Med. Therapie,bei dem Betroffenen selbst,-,-,Medikamentengabe oral,-,Internistisch,-,Kreislaufprobleme,-


## <a id='toc1_6_'></a>[💾 data for pivot use](#toc0_)

In [71]:
if ENABLE_UPDATE:
    # # * only show cols w/ >1 value
    df_csv_short = df_csv.loc[:, df_csv.apply(pd.Series.nunique) > 1]

    df_csv.to_csv(Path(dir_data_out/"df_csv.csv"),sep=";", index=False, encoding="utf-8-sig")
    df_csv_short.to_csv(Path(dir_data_out/"df_csv_short.csv"),sep=";", index=False, encoding="utf-8-sig")
    df_csv_condensed.sort_index(axis=1).to_csv(Path(dir_data_out/"df_csv_condensed.csv"),sep=";", index=False, encoding="utf-8-sig")